In [207]:
import rdflib
import numpy as np
import pylab
import os
import matplotlib.pyplot as plt
import matplotlib
import FuXi
from pandas import *
import pandas
from IPython.lib.pretty import pprint
from rdflib import Variable, Namespace
from rdflib import plugin,RDF,RDFS,URIRef,URIRef,Literal,Variable
from tabulate import tabulate

In [62]:
#Getting RDF datasets 
#Parsing Datasets
g = rdflib.Graph()
g.parse("datasets/the-counted.rdf")
g.parse("datasets/US-Population.rdf")
pop = read_csv('datasets/US-Population')
g.serialize(format='pretty-xml')
print("graph has %s statements." % len(g))

graph has 11995 statements.


In [171]:
#Getting Police Death Statistics 
#Displaying RDF Model
for s, p, o in g:
   if (s, p, o) not in g:
       raise Exception("It better be!")
#Printing Raw RDF DCAT Data
print( g.serialize(format='n3') )
#print out head out data
pop.head(2)

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterm: <http://purl.org/dc/terms/> .
@prefix ds: <http://opendata.socrata.com/resource/hibn-q67x/> .
@prefix ds1: <http://opendata.socrata.com/resource/ru3q-edzk/> .
@prefix dsbase: <http://opendata.socrata.com/resource/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix geo: <http://www.w3.org/2003/01/geo/wgs84_pos#> .
@prefix ods: <http://open-data-standards.github.com/2012/01/open-data-standards#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix socrata: <http://www.socrata.com/rdf/terms#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://opendata.socrata.com/resource/hibn-q67x/1> a dsbase:hibn-q67x ;
    ds:age "18" ;
    ds:armed "No" ;
    ds:cause "Unknown" ;
    ds:city "Key Largo" ;
    ds:day "1" ;
    ds:gender "Male" ;
 

,SUMLEV,REGION,DIVISION,STATE,SEX,ORIGIN,RACE,AGE,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014
0,State and/or Statistical Equivalent,South,East South Central,ALABAMA,Total,Total,White,0,40144,40144,40454,40863,40000,37943,37515
1,State and/or Statistical Equivalent,South,East South Central,ALABAMA,Total,Total,White,1,40184,40184,40109,40587,40968,40073,38114


In [172]:
#Grouping Population By Race and Total Count for each Race
races = pop.groupby('RACE')[('POPESTIMATE2014')].sum().order()
#Pretty printing data 
population = DataFrame(races[::-1])
print '                     US population for each Race                                     '
print '----------------------------------------------------------------------------'
#pprint(population)
headersp = ['RACE' , 'TOTAL POPULATION']
print tabulate(population,headers, tablefmt="fancy_grid",floatfmt=".0f")

                     US population for each Race                                     
----------------------------------------------------------------------------
╒═════════════════╤════════════╕
│ NAME            │     GENDER │
╞═════════════════╪════════════╡
│ White           │ 1014900744 │
├─────────────────┼────────────┤
│ Black           │  182689000 │
├─────────────────┼────────────┤
│ Asian           │   81001000 │
├─────────────────┼────────────┤
│ American Indian │   26130392 │
├─────────────────┼────────────┤
│ Native Hawaiian │    5876704 │
╘═════════════════╧════════════╛


In [173]:
#Displaying the names, gender, and ethnicity of people killed by police so far in 2015
#Running SPARQL Query to aggregaete RDF data
policeKillings = g.query("""
    SELECT DISTINCT ?name ?gender ?raceethnicity ?age ?armed ?state  
    WHERE{  
            ?s ds:name ?name.
            ?s ds:gender ?gender.
            ?s ds:raceethnicity ?raceethnicity.
            ?s ds:age ?age.
            ?s ds:armed ?armed.
            ?s ds:state ?state.} 
    ORDER BY ASC(?state)
    LIMIT 50
    """)
#Print out Query
print("-----------------------------People Who got Killed by Police in 2015------------------------\n")
headers = ['NAME', 'GENDER','ETHNICITY','AGE','ARMED','STATE']
#deathsByRace = tabulate(policeKillings,headers, tablefmt="fancy_grid")
deathsByRace = tabulate(policeKillings,headers)
print deathsByRace

-----------------------------People Who got Killed by Police in 2015------------------------

NAME                   GENDER    ETHNICITY          AGE  ARMED               STATE
---------------------  --------  ---------------  -----  ------------------  -------
Larry Kobuk            Male      Native American     33  No                  AK
John Whittaker         Male      Hispanic/Latino     33  Firearm             AK
Anthony Briggs         Male      Black               36  Knife               AL
Douglas Harris         Male      Black               77  Firearm             AL
Fletcher Stewart       Male      White               46  Firearm             AL
Shane Watkins          Male      White               39  Other               AL
A'donte Washington     Male      Black               16  Firearm             AL
Stanley Grant          Male      Black               38  Firearm             AL
William 'Rusty' Smith  Male      White               53  Firearm             AL
Gilbert Fleury    

In [174]:
#Displaying the number of deaths for each race
#Running SPARQL Query to aggregaete RDF data
tbr = g.query("""
    SELECT DISTINCT ?raceethnicity (COUNT(?raceethnicity) AS ?rCount)
    WHERE{?s ds:raceethnicity ?raceethnicity.}
    GROUP BY ?raceethnicity 
    ORDER BY DESC(?rCount)
    """)
#Print out Query
print("-----------------------------Total Deaths by Police ordered by Race------------------------\n")
headers2 = ['RACE','TOTAL DEATHS' ]
print tabulate(tbr,headers2,tablefmt="fancy_grid")

-----------------------------Total Deaths by Police ordered by Race------------------------

╒════════════════════════╤════════════════╕
│ RACE                   │   TOTAL DEATHS │
╞════════════════════════╪════════════════╡
│ White                  │            253 │
├────────────────────────┼────────────────┤
│ Black                  │            143 │
├────────────────────────┼────────────────┤
│ Hispanic/Latino        │             77 │
├────────────────────────┼────────────────┤
│ Asian/Pacific Islander │             11 │
├────────────────────────┼────────────────┤
│ Unknown                │              9 │
├────────────────────────┼────────────────┤
│ Native American        │              5 │
├────────────────────────┼────────────────┤
│ Arab-American          │              2 │
╘════════════════════════╧════════════════╛


In [213]:
#Verifying Data with csv file
tc = read_csv('datasets/the-counted')
totalDeaths = tc.groupby(['raceethnicity']).size().order()
td = DataFrame(totalDeaths[::-1])
print tabulate(td,headers2,tablefmt="fancy_grid")
print"    COMPARING GRAPHS FOR TOTAL RATES\n                 "
print tabulate(population,headersp, tablefmt="fancy_grid",floatfmt=".0f")
ttr = DataFrame()
print"\n---------------------OVERALL TOTAL DEATH RATE by PEOPLE KILLED BY THE POLICE------------------------"
print tabulate([["RACE","TOTAL DEATH RATE BY POLICE"],["White",3.2],["Black",9.6],["Asian",1.23],["American Indian",4.97],["Native Hawaiian",1.7]]
               ,headers="firstrow",floatfmt=".2f",tablefmt="fancy_grid")

╒════════════════════════╤════════════════╕
│ RACE                   │   TOTAL DEATHS │
╞════════════════════════╪════════════════╡
│ White                  │            334 │
├────────────────────────┼────────────────┤
│ Black                  │            176 │
├────────────────────────┼────────────────┤
│ Hispanic/Latino        │            100 │
├────────────────────────┼────────────────┤
│ Unknown                │             46 │
├────────────────────────┼────────────────┤
│ Asian/Pacific Islander │             13 │
├────────────────────────┼────────────────┤
│ Native American        │              8 │
├────────────────────────┼────────────────┤
│ Arab-American          │              3 │
╘════════════════════════╧════════════════╛
    COMPARING GRAPHS FOR TOTAL RATES
                 
╒═════════════════╤════════════════════╕
│ RACE            │   TOTAL POPULATION │
╞═════════════════╪════════════════════╡
│ White           │         1014900744 │
├─────────────────┼──────────────